In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [ ]:
from generator import FunctionCallGenerator
from evaluator import FunctionCallEvaluator

generator = FunctionCallGenerator(config)
evaluator = FunctionCallEvaluator()

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from promptflow.client import PFClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
       subscription_id=config["AZURE_SUB_ID"],
       resource_group_name=config["AZURE_RESOURCE_GROUP"],
       workspace_name=config["AZURE_AI_PROJECT_NAME"],
       credential=DefaultAzureCredential()
)

pf_client = PFClient()
local_path = "function_call_eval"
pf_client.flows.save(entry=FunctionCallEvaluator, path=local_path)

evaluator_name = "FunctionCallEvaluator"

custom_evaluator = Model(
    path=local_path,
    name=evaluator_name,
    description="A custom evaluator for function call accuracy.",
)

ml_client.evaluators.create_or_update(custom_evaluator)

In [ ]:
generator("gradually warm the house starting at 6am tomorrow")

In [ ]:
from azure.ai.evaluation import evaluate

result = evaluate(
    data = "../../docs/dataset.jsonl",
    target = generator,
    evaluators = {
        "function_call_compare": evaluator
    },
    evaluator_config = {
        "default": {
            "column_mapping": {
                "intent": "${data.intent}",
                "expected_calls": "${data.expected_calls}",
                "actual_calls": "${target.actual_calls}"
            } 
        }
    },
    azure_ai_project = {
        "subscription_id": config["AZURE_SUB_ID"],
        "resource_group_name": config["AZURE_RESOURCE_GROUP"],
        "project_name": config["AZURE_AI_PROJECT_NAME"]
    },
    output_path="./output.json"
)

result["studio_url"]